<a href="https://colab.research.google.com/github/honey-kolleri/Recomendation_System_Application/blob/main/Netflix_Movie_Recomendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Step:1 Install The Required Packages
!pip install pandas numpy matplotlib seaborn scikit-learn

In [ ]:
#Step: 2 Import The Required Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Step:3 Load The Netflix_titles DataSet
df=pd.read_csv('netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
# Step 4: Combined The Key Columns with explicit string conversion
# We use .fillna('') on each column to ensure no 'nan' strings are created
df['Combine_Text'] = (
    df['title'].fillna('') + ' ' +
    df['director'].fillna('') + ' ' +
    df['cast'].fillna('') + ' ' +
    df['listed_in'].fillna('') + ' ' +
    df['description'].fillna('')
).str.lower() # Converting to lowercase helps with matching

# Double-check: Replace any remaining stray nulls in the final column
df['Combine_Text'] = df['Combine_Text'].fillna('')
# Optional: Verifying there are no nulls left
print(f"Number of nulls in Combine_Text: {df['Combine_Text'].isnull().sum()}")

Number of nulls in Combine_Text: 0


In [ ]:
# Step 5: Converting the Combine_Text Column into Vectors
tfidf_object = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range=(1, 2))

# Convert Text to Vectors (Keep as sparse matrix for speed/memory)
tfidf_matrix = tfidf_object.fit_transform(df['Combine_Text'])

# Skiping the creation of the tdidf_df (the DataFrame) to save RAM.
# The recommendation function will use tfidf_matrix instead.

In [ ]:
#Step:6 Create Cosine_Similarity For Recommending The Similar Product
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Step 7: Recommendation Function
def recommend_movie(query, k=5):
    # Convert query text to vector
    query_vec = tfidf_object.transform([query])

    # Calculate similarity between query and all movies
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # Rank the movies and get top indices
    top_indices = similarity.argsort()[-k:][::-1]

    # Return the title, genre, and description of top matches
    result = df.iloc[top_indices][['title', 'type', 'release_year', 'listed_in', 'description']]
    return result
    # Usage
search_query = input('Enter a movie title, actor, or genre you like: ')
recommend_movie(search_query, 10)

Enter a movie title, actor, or genre you like: romance movies


,title,type,release_year,listed_in,description
287,I Need Romance,TV Show,2021,"International TV Shows, Romantic TV Shows, TV ...",A workaholic who lost interest in romance reun...
1090,Story of Kale: When Someone's in Love,Movie,2020,"Dramas, International Movies, Romantic Movies","After leaving a toxic relationship, Dinda emba..."
986,Kocan Kadar Konus 2: Dirilis,Movie,2016,"Comedies, International Movies, Romantic Movies","Efsun has found Mr. Right in Sinan, and their ..."
7351,Love Station,Movie,2011,"Comedies, International Movies, Romantic Movies",Single coworkers at a radio station are brough...
3070,Posesif,Movie,2017,"Dramas, International Movies, Romantic Movies",A student diver risks her scholastic future an...
6908,Hadi İnşallah,Movie,2014,"Comedies, International Movies, Romantic Movies",A young woman recovering poorly from a bad rom...
3486,Malaal,Movie,2019,"Dramas, International Movies, Romantic Movies",Class divides become thorns in the budding rom...
2286,Pyaar Ka Punchnama 2,Movie,2015,"Comedies, Dramas, International Movies",Three young bachelors find the reality of rela...
2878,Couple of Days,Movie,2016,"Dramas, International Movies, Romantic Movies","Three couples, each in different phases of rom..."
5374,Amar,Movie,2017,"Dramas, International Movies, Romantic Movies",Young Laura and Carlos experience the intensit...
